Homework 4

Anushka Menon and Yashavee Goel

In [229]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
import math

In [230]:
m_pi = np.array([0.176, 0.234, 0.260, 0.284, 0.324])
f = np.array([960, 1025, 1055, 1085, 1130])
delta_f = np.array([25, 20, 15, 10, 8])

def func(m_pi, Phi):
    return Phi[0]+Phi[1]*m_pi+Phi[2]*m_pi**2
    
def chi_sq(f, delta_f):
    for i in range(5):
        return np.sum(0.5*(f[i]-func(m_pi, Phi))/delta_f[i])

In [235]:
 def P_dot(Phi, m_pi, f, delta_f):
    p_dot = []
    for i in range(3):
        sum = 0
        for j in range(5): 
            sum += ((beta*(f[j]-func(m_pi, Phi))*m_pi[j]**i)/delta_f[j])
        p_dot.append(sum)   
    return np.array(p_dot)  
                                
                            
def leapfrog(N_md, P_0, Phi_0, J, beta, h, N):
    eps = 1/N_md
    P = P_0.astype("float")
    Phi = Phi_0.astype("float")
    Phi += eps/2*Phi
    for i in range(N_md-1):
        P += eps*P_dot(Phi, m_pi, f, delta_f)
        Phi += eps*P
    P -= eps*P_dot(Phi, m_pi, f, delta_f)
    Phi += eps/2*Phi
    return P, Phi
        
def Hamiltonian(P, Phi, J, beta, h, N):
    return P[0]**2/2+P[1]**2/2+P[2]**2/2+chi_sq(f, delta_f)

In [236]:
P_0 = np.array([1,2,3])
Phi_0 = np.array([1,2,3])
beta = 1000
N_md = 20
N_cfg = 10000
h = 0.5
N = 20
J = 1/N
Phi = np.array([800,800,600])

In [237]:
diff = []
range_1 = range(1,100)

for N_md in range_1:
    arr = leapfrog(N_md, P_0, Phi_0, J, beta, h, N)
    P_prime = arr[0]          # We need final values of p_f and phi_f to calculate H_f
    Phi_prime = arr[1]
    H_f = Hamiltonian(P_0, Phi_0, J, beta, h, N)
    H_0 = Hamiltonian(P_prime, Phi_prime, J, beta, h, N)
    
#     The following is the quantity we want on y-axis
    diff.append(abs((H_f-H_0)/H_0))    

ValueError: operands could not be broadcast together with shapes (3,) (3,5) (3,) 

In [234]:
plt.figure(figsize=(12,10))
plt.yscale('log')
plt.grid()
plt.xlabel("$N_{\md}$")
plt.ylabel("$H_{\f}$ - $H_{\o}$ / $H_{\o}$")
plt.title("Convergence of leap frog integrator", size =12)
plt.plot(range_1, diff,'o')
plt.show()

ValueError: x and y must have same first dimension, but have shapes (99,) and (0,)

Error in callback <function _draw_all_if_interactive at 0x7f3738102670> (for post_execute):


ValueError: 
H_{}
   ^
Expected {accent | symbol | function | operatorname | group | frac | dfrac | binom | genfrac | overset | underset | sqrt | overline}, found '\x0c'  (at char 3), (line:1, col:4)

ValueError: 
H_{}
   ^
Expected {accent | symbol | function | operatorname | group | frac | dfrac | binom | genfrac | overset | underset | sqrt | overline}, found '\x0c'  (at char 3), (line:1, col:4)

<Figure size 1200x1000 with 1 Axes>

Answer 3 : HMC Algorithm

In [223]:
def markov_chain(Phi_i,N_cfg,N_md,beta,J,h,N):
    chain = []
    chain.append(Phi_i)
    for i in range(N_cfg):
        p = np.random.normal(0,1)  # Random sample
        new = leapfrog(N_md, P, chain[-1], J, beta, h, N) # Integrating leapfrog algorithm
        prob = np.exp(Hamiltonian(P,chain[-1], J, beta, h, N)-Hamiltonian(new[0], new[1], J, beta, h, N))
#  Given probability distribution function defined above
        if prob > np.random.uniform(0,1):   # Accept/reject
            chain.append(new[1])  
        else:                           
            chain.append(chain[-1]) 
    return chain